In [1]:
import os
!pwd

/config/workspace/experiments


In [2]:
os.chdir("../")

In [3]:
%pwd

'/config/workspace'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [5]:
from ml_ops.constants import *
from ml_ops.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [7]:
import os
from ml_ops import logger
import pandas as pd

In [8]:
class DataValidation:
    def __init__(self, config:DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool : 
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            # all_schema = self.config.all_schema.keys()
            all_schema = self.config.all_schema

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    break # No need to continue checking if a column is missing in the schema
                else:
                    expected_dtype = all_schema[col]
                    actual_dtype = data[col].dtype
                    # print(f"Expected dtype is {expected_dtype} & Actual dtype is {actual_dtype}")

                    if expected_dtype != actual_dtype:
                        validation_status = False
                        with open(self.config.STATUS_FILE, 'w') as f:
                            f.write(f"Validation status: {validation_status}, "f"Column '{col}' has incorrect datatype. " f"Expected: {expected_dtype}, Actual: {actual_dtype}")
                        break  # No need to continue checking if a datatype is incorrect

                    else:
                        validation_status = True

                    
            if validation_status is None:
                validation_status = True  # If all columns are present and have correct datatypes

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

    
        except Exception as e:
            raise e 

In [9]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-02-01 15:13:27,985: INFO: common: yaml file: config/config.yaml loaded succesfullly] 
[2024-02-01 15:13:27,987: INFO: common: yaml file: params.yaml loaded succesfullly] 
[2024-02-01 15:13:27,991: INFO: common: yaml file: schema.yaml loaded succesfullly] 
[2024-02-01 15:13:27,992: INFO: common: created directory at: artifacts] 
[2024-02-01 15:13:27,993: INFO: common: created directory at: artifacts/data_validation] 
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Expected dtype is float64 & Actual dtype is float64
Ex

In [10]:
d = pd.read_csv('experiments/winequalityN.csv')
d.dtypes[1]

dtype('float64')